### Documentation: 

#### Overall
The main goal of this project is to convert multiple excel data files to appended rows in a SQL database. This will operate via the following procedure: 

1. Python/SQLite connection to SQL database. 
2. Clean each csv/excel file via python
3. Upload and replace associated table in SQLfile 

#### GSM Member Instructions after download and save: 
1. From August Dashboard xlsx file take out all spaces manually (rename)


#### Location path
C:\Users\ckato\Documents\GitHub\metals-research\data

#### To upload into Power BI, install driver first here: 
http://www.ch-werner.de/sqliteodbc/
    

In [2]:
from sqlalchemy import create_engine
from sqlalchemy import(Table, Column, String, Integer, Boolean)
from sqlalchemy import MetaData, Table

import pandas as pd
# https://campus.datacamp.com/courses/introduction-to-relational-databases-in-python/creating-and-manipulating-your-own-databases?ex=1
    
# Missing decimal

In [3]:
# Connecting to Engine
engine = create_engine('sqlite:///metals_dashboard.sqlite')

connection = engine.connect()

In [4]:
# What's in the database already?
print(engine.table_names())

['battery_cap']


### Iteratively Updating

Steps to creating database

1. load csv file and create pandas dataframe
2. connect to sql server
3. concatenate subset of csv file to the end of the existing sql table associated with that csv file 



In [5]:
# Conversion of FINAL-SQL file to pandas for dates range

# !?? NOTE WILL CONVERT dataframe of query (date file only) to pandas 
# pandas extraction of NEW ROW INDICES ONLY to be appended to all sql 
# table files

# create new sql statement to insert/append where clause from indices 

df = pd.read_sql_query('SELECT * from battery_cap;', connection)
df.head()

#stmt = select([census])
#conn = sqlite3.connect("flights.db")





,index,Megafactory,Region,GWh 2018,GWh 2023,GWh 2028,Date
0,1,"AESC, Zama, Japan",Asia (excl China),2.6,2.6,2.6,2000-01-01 00:00:00.000000
1,2,"LEJ, Shiga, Japan",Asia (excl China),2,2,4,2000-01-01 00:00:00.000000
2,3,"LG Chem, Ochang, Korea",Asia (excl China),12,16,20,2000-01-01 00:00:00.000000
3,4,"Panasonic, Himeji, Japan",Asia (excl China),3,8,20,2000-01-01 00:00:00.000000
4,5,"Panasonic, Kasai, Japan",Asia (excl China),3,5,10,2000-01-01 00:00:00.000000


## Read Raw Data from raw_data_finals folder

Reading and preprocessing/cleaning data

In [203]:
filename = 'raw_data_finals\MetalsDashboard(Aug)LATEST.xlsx'
commodities = pd.read_excel(filename, sheet_name="Commodities Data")


# read census.csv into a dataframe to create pandas dataframe
#commodities = pd.read_csv("raw_data_finals\Commodities.csv")
commodities = commodities.iloc[:, 0:len(commodities.columns)]

commodities.columns = commodities.iloc[0]

#############
df = commodities.iloc[6: ,:]
df = df.iloc[:, 1:]
df = df.rename(columns={"Column1": "Date"})

df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df = df.drop("Fastmarkets - Cobalt High", axis=1)
df = df.drop("Fastmarkets- Cobalt LOW", axis=1)
df = df.drop("Column2", axis=1)

df = df.astype(float)

df.head(10)

,LME Ni cash price,LME Ni inventory (total),LME Ni inventory (briquette),LME Ni inventory (other forms),Critical Inventory,LME Co cash price,LME Cu cash price,Comex Cu future,Comex Cu future2,LME Al cash price
Date,,,,,,,,,,
2015-01-02,14756.0,414900.0,165720.0,249180.0,100000.0,31619.0,6321.00,272.65,6005.506608,1805.25
2015-01-05,15126.0,414756.0,165972.0,248784.0,100000.0,31365.0,6213.50,267.45,5890.969163,1787.25
2015-01-06,15191.0,415338.0,166506.0,248832.0,100000.0,31366.0,6221.00,267.30,5887.665198,1755.25
2015-01-07,15483.0,415812.0,167094.0,248718.0,100000.0,31367.0,6182.50,266.10,5861.233480,1761.75
2015-01-08,15486.0,415842.0,167124.0,248718.0,100000.0,31371.0,6175.75,267.30,5887.665198,1811.50
2015-01-09,15235.5,415164.0,166818.0,248346.0,100000.0,31371.0,6166.50,265.45,5846.916300,1789.75
2015-01-12,15029.0,414732.0,166554.0,248178.0,100000.0,31367.5,6101.00,262.55,5783.039648,1791.00
2015-01-13,14585.0,416436.0,167244.0,249192.0,100000.0,30769.0,5939.50,254.00,5594.713656,1779.25
2015-01-14,14241.0,418320.0,167142.0,251178.0,100000.0,30769.5,5619.00,241.20,5312.775330,1765.50


### Drop Table

In [175]:
result_proxy = connection.execute('DROP table tester1;')

# fetches a result set as python object
#results = result_proxy.fetchall()

# What's in the database already?
print(engine.table_names())

[]


### Create Table

In [185]:
df.to_sql('commodities', con=engine, if_exists='replace', index = True)

results = engine.execute("SELECT * FROM commodities").fetchall()

#result_proxy = connection.execute(stmt)
for item in results:
    #print(item.keys())
    print(item.values())


['2015-01-02 00:00:00.000000', 14756.0, 414900.0, 165720.0, 249180.0, 100000.0, 31619.0, 6321.0, 272.65]
['2015-01-05 00:00:00.000000', 15126.0, 414756.0, 165972.0, 248784.0, 100000.0, 31365.0, 6213.5, 267.45]
['2015-01-06 00:00:00.000000', 15191.0, 415338.0, 166506.0, 248832.0, 100000.0, 31366.0, 6221.0, 267.3]
['2015-01-07 00:00:00.000000', 15483.0, 415812.0, 167094.0, 248718.0, 100000.0, 31367.0, 6182.5, 266.1]
['2015-01-08 00:00:00.000000', 15486.0, 415842.0, 167124.0, 248718.0, 100000.0, 31371.0, 6175.75, 267.3]
['2015-01-09 00:00:00.000000', 15235.5, 415164.0, 166818.0, 248346.0, 100000.0, 31371.0, 6166.5, 265.45]
['2015-01-12 00:00:00.000000', 15029.0, 414732.0, 166554.0, 248178.0, 100000.0, 31367.5, 6101.0, 262.55]
['2015-01-13 00:00:00.000000', 14585.0, 416436.0, 167244.0, 249192.0, 100000.0, 30769.0, 5939.5, 254.0]
['2015-01-14 00:00:00.000000', 14241.0, 418320.0, 167142.0, 251178.0, 100000.0, 30769.5, 5619.0, 241.2]
['2015-01-15 00:00:00.000000', 14413.5, 418332.0, 167100.0,

## SQL Body

SQL statement below needs to be able to merge the most recent columns of the pandas csv dataframe (**NOTE: Using a where clause that only allows for the NEW/UNIQUE dates to be selected**) to a the SQL database 

- After that, execute the statement and fetch (as read-only to check) 

In [186]:
print(engine.table_names())

['commodities']


In [190]:
# How to convert SQL table to pandas dataframe


df = pd.read_sql_query('SELECT * from commodities;', connection)

df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df = df.astype(float)
df

,LME Ni cash price,LME Ni inventory (total),LME Ni inventory (briquette),LME Ni inventory (other forms),Critical Inventory,LME Co cash price,LME Cu cash price,Comex Cu future
Date,,,,,,,,
2015-01-02,14756.00,414900.0,165720.0,249180.0,100000.0,31619.0,6321.00,272.65
2015-01-05,15126.00,414756.0,165972.0,248784.0,100000.0,31365.0,6213.50,267.45
2015-01-06,15191.00,415338.0,166506.0,248832.0,100000.0,31366.0,6221.00,267.30
2015-01-07,15483.00,415812.0,167094.0,248718.0,100000.0,31367.0,6182.50,266.10
2015-01-08,15486.00,415842.0,167124.0,248718.0,100000.0,31371.0,6175.75,267.30
2015-01-09,15235.50,415164.0,166818.0,248346.0,100000.0,31371.0,6166.50,265.45
2015-01-12,15029.00,414732.0,166554.0,248178.0,100000.0,31367.5,6101.00,262.55
2015-01-13,14585.00,416436.0,167244.0,249192.0,100000.0,30769.0,5939.50,254.00
2015-01-14,14241.00,418320.0,167142.0,251178.0,100000.0,30769.5,5619.00,241.20


In [4]:
# OLD / BROKEN
values_list = []

df = pd.read_csv('data\city.csv')
print(df.shape)

for col in df:
    print(col)
    
    data = {'name': col[0], 'population': col[1]}
    #print(data)
    #values_list.append(data)

#print(values_list)

# SQL ALCHEMY NOO

# Access table with python
# metadata reads daabase and builds a SQL alchemy table object

metadata = MetaData()

sqlalchemy_table = Table('census', metadata, autoload=True,
           autoload_with = engine)

print(repr(sqlalchemy_table))

(276, 2)
name
population


In [205]:
connection.close()